# LAB VISUAL ETL TRANSFORMATION SCRIPT

In [ ]:
import os
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from data_task_helpers import something
something()
from api_data_task_executioner.data_task_tools import initialise_data_task, DataTaskEnvironment, find_json_arg  # noqa: E402
import nest_asyncio
import sys
import time

nest_asyncio.apply()
params = {}
environment: DataTaskEnvironment = None

if __name__ == "__main__":
    environment = initialise_data_task("Tille Lab Transformation Task", params=params)
    json_args = find_json_arg(sys.argv)
    params["sleep_time"] = json_args.get("sleep_time", 1.5)

time.sleep(params["sleep_time"])

## Mysql Server Configuration

In [ ]:
default_mysql_config = {
    "MYSQL_USER": "{mysql_user}",
    "MYSQL_PASSWORD": "{mysql_pasword}",
    "MYSQL_HOST": "localhost",
    "MYSQL_DATABASE_NAME": "labdashdb",
    "MYSQL_PORT": "3306",
}

mysql_url = f"mysql+pymysql://{default_mysql_config['MYSQL_USER']}:{default_mysql_config['MYSQL_PASSWORD']}@{default_mysql_config['MYSQL_HOST']}:{default_mysql_config['MYSQL_PORT']}/{default_mysql_config['MYSQL_DATABASE_NAME']}"

try:
    mysql_conn = create_engine(mysql_url)
    environment.log_message(f"Connected to MySQL database: {default_mysql_config['MYSQL_DATABASE_NAME']}")
    time.sleep(params["sleep_time"])
except Exception as e:
    environment.log_error(f"Error connecting to MySQL: {e}")
    sys.stdout.flush()
    sys.exit(1) 

## Mssql Server Configuration

In [ ]:
mssql_config = {
    "server": "localhost",
    "database": "master",
    "username": "{mssql_user}",
    "password": "{mssql_pasword}",
}

mssql_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={mssql_config['server']};"
    f"DATABASE={mssql_config['database']};"
    f"UID={mssql_config['username']};"
    f"PWD={mssql_config['password']};",
    autocommit=True
)

mssql_cursor = mssql_conn.cursor()

## Create lab_visual_analysis database

In [ ]:
def create_lab_visual_analysis_database():
    try:
        mssql_cursor.execute(
            f"""
                IF EXISTS (SELECT * FROM sys.databases WHERE name = 'lab_visual_analysis') 
                BEGIN
                    DROP DATABASE lab_visual_analysis;
                END
                CREATE DATABASE lab_visual_analysis;
            """
        )

        mssql_conn.commit()
        mssql_config["database"] = "lab_visual_analysis"
        environment.log_message("MSSQL database lab_visual_analysis Created.")
        time.sleep(params["sleep_time"])
    except Exception as e:
        environment.log_error(f"Error creating database: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

create_lab_visual_analysis_database()

## Create Schemas

In [ ]:
def create_schemas():
    environment.log_message("Creating Schemas...")
    try:
        schemas = ['source', 'derived', 'final', 'z', 'dbo']
        for schema in schemas:
            mssql_cursor.execute(
                f"""
                USE lab_visual_analysis;

                IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = '{schema}')
                BEGIN
                    EXEC('CREATE SCHEMA {schema}');
                END

                -- Grant CONTROL permission on the schema to the user
                GRANT CONTROL ON SCHEMA::{schema} TO [{mssql_config['username']}];

                -- Optional: Grant db_ddladmin role to allow the user to create/modify schemas
                EXEC sp_addrolemember 'db_ddladmin', '{mssql_config['username']}';
                """
            )
        mssql_conn.commit()
        time.sleep(params["sleep_time"])
    except Exception as e:
        environment.log_error(f"Error creating schemas: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

create_schemas()

## Create Tables in the lab_visual_analysis database

In [ ]:
def create_tables():
    environment.log_message("Creating Tables...")
    table_creation_queries = [
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Facilities' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Facilities (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                HfrCode NVARCHAR(50),
                Name NVARCHAR(255),
                Region NVARCHAR(255),
                District NVARCHAR(255),
                Council NVARCHAR(255)
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Device_Logs' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Device_Logs (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                DeviceName NVARCHAR(255),
                DeviceCode NVARCHAR(50),
                DateBrokenDown DATETIME2,
                DateReported DATETIME2,
                DateFixed DATETIME2,
                BreakDownReason NVARCHAR(255)
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Commodity_Transactions' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Commodity_Transactions (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                CommodityName NVARCHAR(255),
                CommodityCode NVARCHAR(50),
                BatchNumber NVARCHAR(50),
                TransactionDate DATETIME2,
                ExpireDate DATETIME2,
                TransactionType NVARCHAR(50),
                TransactionQuantity INT
            );
        """,
        """
            IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'tbl_Sample' AND schema_id = SCHEMA_ID('source'))
            CREATE TABLE source.tbl_Sample (
                Id INT IDENTITY(1,1) PRIMARY KEY,
                sampletrackingid NVARCHAR(50),
                LabHfrCode NVARCHAR(50),
                HubHfrCode NVARCHAR(50),
                EntryModality NVARCHAR(50),
                SampleType NVARCHAR(255),
                TestName NVARCHAR(255),   
                SampleQualityStatus NVARCHAR(50),
                Results NVARCHAR(255),
                SampleRejectionReason NVARCHAR(255),
                DeviceName NVARCHAR(255),
                DeviceCode NVARCHAR(50),
                CollectionDate DATETIME2,
                ReceivedDate DATETIME2,
                TestDate DATETIME2,
                AuthorisedDate DATETIME2,
                DispatchDate DATETIME2
            );
        """
    ]

    for query in table_creation_queries:
        try:
            mssql_cursor.execute(query)
        except Exception as e:
            environment.log_error(f"Error creating table: {e}")

    mssql_conn.commit()
    time.sleep(params["sleep_time"])

create_tables()


## Load Facility Data

In [ ]:
def extract_and_insert_facility_data():
    excel_file = "./All_Operating_Health_Facilities_in_Tanzania-Lab-Visual-2021oct22.xlsx"
    
    if not os.path.exists(excel_file):
        environment.log_error(f"Excel file '{excel_file}' not found.")
        return
    
    df = pd.read_excel(excel_file)

    df.rename(columns={"Facility Number": "HfrCode", "Facility Name": "Name"}, inplace=True)
    
    df = df[['HfrCode', 'Name', 'Region', 'District', 'Council']]
    df['Region'] = df['Region'].str.replace("Region", "", regex=True).str.strip()
    
    df.drop_duplicates(subset=['HfrCode'], keep='first', inplace=True)

    if df['HfrCode'].duplicated().sum() > 0:
        environment.log_error("Duplicate HfrCode values detected in DataFrame. Cleaning...")
        df = df.drop_duplicates(subset=['HfrCode'])
    
    insert_query = """
        MERGE INTO source.tbl_Facilities AS target
        USING (VALUES (?, ?, ?, ?, ?)) AS src (HfrCode, Name, Region, District, Council)
        ON target.HfrCode = src.HfrCode
        WHEN MATCHED THEN 
            UPDATE SET Name = src.Name, Region = src.Region, District = src.District, Council = src.Council
        WHEN NOT MATCHED THEN 
            INSERT (HfrCode, Name, Region, District, Council) VALUES (src.HfrCode, src.Name, src.Region, src.District, src.Council);
    """
    
    inserted_rows = 0
    try:
        # params = list(df.itertuples(index=False, name=None))
        # mssql_cursor.executemany(insert_query, params)
        for _, row in df.iterrows():
            mssql_cursor.execute(insert_query, row['HfrCode'], row['Name'], row['Region'], row['District'], row['Council'])
        mssql_conn.commit()
        inserted_rows = len(df)
        environment.log_message(f"{inserted_rows} rows inserted into tbl_Facilities.")
    except Exception as e:
        environment.log_error(f"Error inserting data: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

extract_and_insert_facility_data()
time.sleep(params["sleep_time"])

## Load Sample Data

In [ ]:
def extract_and_insert_sample_data():
    query = """
    SELECT DISTINCT trackingID as sampletrackingid, 
                    facilityHfrID, 
                    sampleType as SampleType, 
                    testName as TestName, 
                    sampleQuality as SampleQualityStatus, 
                    rejectionReason as SampleRejectionReason, 
                    sampleCollectionDate as CollectionDate, 
                    dateReceivedLab as ReceivedDate, 
                    results as Results, 
                    testedDate as TestDate, 
                    resultAuthorisedDate as AuthorisedDate, 
                    resultAuthorisedDate as DispatchDate,
                    testInstrument as DeviceName,
                    NULL as DeviceCode,
                    IF(SUBSTR(trackingID, 1, 4) = 'BC03', 'lab', 'hub') as EntryModality
    FROM tbl_labtests
    WHERE sampleCollectionDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR dateSentLab >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR dateReceivedLab >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR registeredDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR testedDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR resultAuthorisedDate >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    OR dateResultSentHub >= DATE_SUB(CURDATE(), INTERVAL 2 MONTH)
    """
    
    try:
        sample_data = pd.read_sql(query, mysql_conn)
        if sample_data.empty:
            environment.log_error("No sample data found.")
            return

        def extract_hfr_code(row):
            if row['EntryModality'] == 'lab':
                return row['facilityHfrID'], None
            else:
                return None, row['facilityHfrID']

        sample_data[['LabHfrCode', 'HubHfrCode']] = sample_data.apply(extract_hfr_code, axis=1, result_type='expand')

        for _, row in sample_data.iterrows():
            insert_query = """
            INSERT INTO source.tbl_Sample (sampletrackingid, LabHfrCode, HubHfrCode, EntryModality, SampleType, 
                                        TestName, SampleQualityStatus, Results, SampleRejectionReason, DeviceName, 
                                        DeviceCode, CollectionDate, ReceivedDate, TestDate, AuthorisedDate, DispatchDate)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            mssql_cursor.execute(insert_query, row['sampletrackingid'], row['LabHfrCode'], row['HubHfrCode'], 
                                row['EntryModality'], row['SampleType'], row['TestName'], row['SampleQualityStatus'], 
                                row['Results'], row['SampleRejectionReason'], row['DeviceName'], row['DeviceCode'], row['CollectionDate'], 
                                row['ReceivedDate'], row['TestDate'], row['AuthorisedDate'], row['DispatchDate'])
        
        mssql_conn.commit()
        environment.log_message(f"{len(sample_data)} rows inserted into tbl_Sample.")
        time.sleep(params["sleep_time"])

    except Exception as e:
        mssql_conn.rollback()
        environment.log_error(f"Error inserting data: {e}")

extract_and_insert_sample_data()
time.sleep(params["sleep_time"])

## Load Device Logs Data

In [ ]:
def extract_and_insert_device_log_data():
    query = """
        select
            deviceName as DeviceName,
            deviceCode as DeviceCode,
            dateBreakDown as DateBrokenDown,
            dateReported as DateReported,
            dateFixed as DateFixed,
            breakDownReason as BreakDownReason
        from
            instrumentlogs2
    """
    
    try:
        device_logs = pd.read_sql(query, mysql_conn)
        
        insert_query = """
        INSERT INTO source.tbl_Device_Logs (DeviceName, DeviceCode, DateBrokenDown, DateReported, DateFixed, BreakDownReason)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        for _, row in device_logs.iterrows():
            mssql_cursor.execute(insert_query, row['DeviceName'], row['DeviceCode'], row['DateBrokenDown'], row['DateReported'], row['DateFixed'], row['BreakDownReason'])
        
        mssql_conn.commit()
        environment.log_message(f"{len(device_logs)} rows inserted into tbl_Device_Logs.")
    except Exception as e:
        environment.log_error(f"Error fetching or inserting device logs: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)

extract_and_insert_device_log_data()
time.sleep(params["sleep_time"])

## Load Commodity Transaction Data

In [ ]:
def extract_and_insert_commodity_transaction_data():
    query = """SELECT commodityName AS CommodityName, commodityCode AS CommodityCode, batchNo AS BatchNumber, transactionDate AS TransactionDate, 
                    expireDate AS ExpireDate, transactionType AS TransactionType, quantity AS TransactionQuantity FROM commoditytransactions"""
    
    try:
        commodity_transactions = pd.read_sql(query, mysql_conn)
        
        insert_query = """
        INSERT INTO source.tbl_Commodity_Transactions (CommodityName, CommodityCode, BatchNumber, TransactionDate, ExpireDate, TransactionType, TransactionQuantity)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        
        for _, row in commodity_transactions.iterrows():
            mssql_cursor.execute(insert_query, row['CommodityName'], row['CommodityCode'], row['BatchNumber'], row['TransactionDate'],
                                row['ExpireDate'], row['TransactionType'], row['TransactionQuantity'])
        
        mssql_conn.commit()
        environment.log_message(f"{len(commodity_transactions)} rows inserted into tbl_Commodity_Transactions.")
    
    except Exception as e:
        environment.log_error(f"Error fetching or inserting commodity transactions: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)
        
extract_and_insert_commodity_transaction_data()
time.sleep(params["sleep_time"])

## Read the stored procedures file

In [ ]:
def extract_stored_procedures_from_file():
    stored_procedures_file = "./create-stored-procedures.sql"
    if os.path.exists(stored_procedures_file):
        environment.log_message("Adding SP...")
        with open(stored_procedures_file, "r") as file:
            sql_commands = file.read()

        sql_batches = sql_commands.split("GO")

        for batch in sql_batches:
            batch = batch.strip()

            if not batch or batch.startswith("--") or batch.startswith("/*"):
                continue

            if batch:
                try:
                    mssql_cursor.execute(batch)
                    mssql_conn.commit()
                except Exception as e:
                    environment.log_error(f"Error loading batch: {e}")
                    mssql_conn.rollback()
                    sys.stdout.flush()
                    sys.exit(1)
    else:
        environment.log_error(f"SQL file '{stored_procedures_file}' not found.")
        sys.stdout.flush()
        sys.exit(1)

extract_stored_procedures_from_file()

## Execute Data Transformation

In [ ]:
def execute_stored_procedures():
    try:
        sp_main = ["dbo.sp_data_processing", "z.sp_data_processing", "derived.sp_data_processing", "final.sp_data_processing"]
        for sp in sp_main:
            sp = f"EXEC {sp}"
            mssql_cursor.execute(sp)
            
            while mssql_cursor.nextset():
                pass
        mssql_conn.commit()

    except Exception as e:
        environment.log_error(f"Error executing stored procedure: {e}")
        mssql_conn.rollback()
        sys.stdout.flush()
        sys.exit(1)
    finally:
        if 'mssql_conn' in locals():
            mssql_cursor.close()
            mssql_conn.close()
            mysql_conn.dispose()

execute_stored_procedures()


In [ ]:
time.sleep(params["sleep_time"])
environment.log_message('Tille Lab Transformation completed!')